In [4]:
import pandas as pd
import os
from dotenv import load_dotenv
import warnings
import tiktoken  # 추가된 부분

# ✅ LangGraph 및 LangChain 관련 모듈
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

# ✅ AI Assistant LangGraph Class Import
from utils.ai_agent import AIAnalysisAssistant  # ai_agent.py에 해당 클래스 정의

warnings.filterwarnings('ignore')
load_dotenv()  # 환경 변수 로드

# ✅ OpenAI API Key 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0,)

dir_output = "output"

dist_file = "월별_가입이력_dummy_data2.csv"
coverage_dist_df = pd.read_csv(dist_file)
df_cust = coverage_dist_df.copy()

dist_file = "고객가입담보이력.csv"
coverage_dist_enroll = pd.read_csv(dist_file)
df_enroll_history = coverage_dist_enroll.copy()

In [ ]:
import pandas as pd
import numpy as np

# 엑셀 파일 경로
excel_path = "output/eda_summary.xlsx"

# 엑셀 파일의 모든 시트 읽기
sheets_dict = pd.read_excel(excel_path, sheet_name=None)

# OpenAI의 tiktoken 사용 (GPT-4 기준)
tokenizer = tiktoken.get_encoding("cl100k_base")

# 각 시트에서 개별적으로 데이터프레임을 분리하여 처리
df_list = {}

# EDA 결과를 데이터프레임에 할당
for sheet_name, df in sheets_dict.items():
    # '데이터프레임명' 컬럼이 존재하는 경우, 개별 데이터프레임으로 분리
    if "데이터프레임명" in df.columns:
        for df_name in df["데이터프레임명"].unique():
            df_list[df_name] = df[df["데이터프레임명"] == df_name].copy()
    else:
        df_list[sheet_name] = df.round(1).copy()  # 일반적인 시트명 기준 데이터 저장


# 데이터프레임 개별 처리 반복
for df_name, df in df_list.items():
    print(f"\n=== 📊 EDA 결과: {df_name} ===")
    display(df.head())  # 상위 5개 행 출력

    ## 📌 **Step 1: EDA 인사이트 도출**
    prompt_insight = f"""
    ### EDA 결과 인사이트 도출

    제공된 데이터프레임은 EDA(탐색적 데이터 분석) 결과입니다.  
    이 데이터를 기반으로 인사이트를 도출하고, 필요한 경우 데이터 정제(전처리) 방법을 제안하세요.  

    **❗ 중요사항:**  
    1. 제공된 데이터가 없는 항목(예: 결측치 정보가 없는데 결측치 분석을 하려는 경우)은 분석하지 마세요.  
    2. 무조건 모든 항목에 대해 인사이트를 생성하지 말고, 연관이 없는 분석 항목은 제거하세요.  
    3. 단순한 데이터 요약이 아니라, 분석 결과를 기반으로 피드백을 제공합니다.  
    4. 데이터 정제(전처리) 방법을 명확히 제시하세요. (예: 결측치 처리, 이상치 제거, 데이터 타입 변환)  
    """

    dict_df_text = df.to_markdown()
    
    # 토큰 수 계산
    token_count = len(tokenizer.encode(dict_df_text))
    print(f"\n토큰 수: {token_count}")
    
    chain_insight = ChatPromptTemplate.from_messages([
                ("system", prompt_insight),
                ("user","### 분석 결과:\n{dict_df_text}\n\n")
            ]) | llm

    insight_response = chain_insight.invoke({"dict_df_text": dict_df_text}).content
    print(f'[LOG] {df_name} 인사이트 생성 결과\n  {insight_response} ')

    ## 📌 **Step 2: 전처리 코드 자동 생성**
    def generate_preprocessing_code(insight_text, error_message=None):
        prompt_preprocessing = f"""
        ### 데이터 전처리 코드 자동 생성

        다음은 탐색적 데이터 분석(EDA) 결과를 기반으로 도출된 인사이트입니다:  

        {insight_text}  

        위 인사이트를 바탕으로 **Python 전처리 코드**를 생성하세요.  
        ❗ **중요: 아래 지침을 반드시 따르세요.**
        
        🔹 **결측치 처리 (Missing Values)**
        - **단순 삭제 금지:** 데이터 손실을 방지하기 위해 결측치는 제거하지 않고 **대체(fillna)하는 방법을 우선적으로 고려**하세요.
        - **대체 방법:** 
          - **범주형 변수**: 최빈값(`mode()`) 또는 `'알 수 없음'`으로 대체.
          - **연속형 변수**: 평균(`mean()`) 또는 중앙값(`median()`)으로 대체.
        - **예외적으로 삭제가 필요한 경우:** 결측 비율이 50% 이상인 컬럼은 삭제 가능.

        🔹 **이상치 처리 (Outlier Handling)**
        - **탐지 방법:** IQR(사분위 범위), 표준편차 기준, z-score 활용 가능.
        - **처리 방법:** 이상치를 제거하거나(`drop`) 적절한 값으로 변환(`clip`, `log 변환` 등)하세요.

        🔹 **데이터 타입 변환 (Data Type Conversion)**
        - **자동 변환 방지:** 숫자형 변수(`int`, `float`)를 범주형(`category`)로 변경할 때, 명확한 기준을 설정하세요.
        - `여부`, `코드`, `등급` 등의 명칭을 가진 변수는 **범주형(`category`)으로 변환**하세요.
        - 문자열(`object`)로 저장된 숫자 값이 있다면 `astype(int)` 또는 `astype(float)`을 사용해 변환하세요.

        🔹 **불필요한 컬럼 삭제 (Feature Selection)**
        - 단순 인덱스, 중복 정보가 포함된 컬럼은 제거 가능.
        - `ID`, `고객번호` 등의 식별자 컬럼은 남겨주세요.

        **출력 형식:**  
        ```python
        import pandas as pd
        import numpy as np

        def preprocess_data(df):
            # 여기에 전처리 코드 삽입
            return df
        ```
        """

        # 오류 발생 시, LLM에게 코드 수정 요청 추가
        if error_message:
            prompt_preprocessing += f"\n\n🔴 **오류 발생:**\n{error_message}\n\n위 오류를 해결할 수 있도록 전처리 코드를 수정하세요."

        chain_preprocessing = ChatPromptTemplate.from_messages([
                    ("system", prompt_preprocessing),
                    ("user","EDA 결과를 바탕으로 데이터 전처리 코드를 생성해주세요. ")
                ]) | llm
        return chain_preprocessing.invoke({}).content

    preprocessing_code = generate_preprocessing_code(insight_response)
    print(f'[LOG] 전처리 코드\n  {preprocessing_code} ')

    ## 📌 **Step 3: 동적 실행을 위해 `exec()` 사용**
    def execute_preprocessing_code(code):
        try:
            exec_code = code.split("```python")[-1].split("```")[0]
            exec(exec_code, globals())  # LLM이 생성한 코드를 실행하여 preprocess_data 함수 정의
            return True  # 실행 성공
        except Exception as e:
            print(f"❌ 전처리 함수 선언 중 오류 발생: {e}")
            return e  # 실행 실패 시 오류 메시지 반환

    # 전처리 함수 선언
    execute_preprocessing_code(preprocessing_code)
    
    ## 📌 **Step 4: 사용자 선택 적용**
    apply_preprocessing = "yes"

    if apply_preprocessing == "yes":
        attempt_count = 0  # 실행 시도 횟수

        while attempt_count < 2:  # 최초 실행 + 1회 재생성 가능
            try:
                display(globals()[df_name].head())
                df_rslt = preprocess_data(globals()[df_name])  # 전처리 적용
                df_rslt.to_csv(f'{dir_output}/stage1/{df_name}.csv')
                print(f"✅ 1차시도 데이터프레임 [{df_name}]의 전처리 코드가 실행되었습니다! | 전처리 이전 데이터 = {globals()[df_name].shape} | 전처리 이후 데이터 = {df_rslt.shape}")
                break  # 실행 성공 시 루프 종료
            except Exception as e:
                error_message = str(e)
                print(f"❌ 1차 시도 : 전처리 코드 실행 중 오류 발생: {error_message}")

                if attempt_count == 0:  # 최초 실행 실패 시 1회만 재생성
                    print("🔄 오류 메시지를 기반으로 전처리 코드 수정 중...")
                    preprocessing_code = generate_preprocessing_code(insight_response, error_message)
                    print(f'[LOG] 수정된 전처리 코드\n  {preprocessing_code} ')

                    execute_preprocessing_code(preprocessing_code)
                    
                    try:
                        df_rslt = preprocess_data(globals()[df_name])  # 전처리 적용
                        df_rslt.to_csv(f'{dir_output}/stage1/{df_name}.csv')
                        print(f"✅ 2차시도 - 데이터프레임 [{df_name}]의 전처리 코드가 실행되었습니다! | 전처리 이전 데이터 = {globals()[df_name].shape} | 전처리 이후 데이터 = {df_rslt.shape}")
                        break  # 실행 성공 시 루프 종료
                    except Exception as e:
                        error_message = str(e)
                        print(f"❌ 2차시도 [{df_name}]의 수정된 코드도 실행 불가. 전처리 실행을 중단합니다.\n Error : {error_message}")

                else:
                    print(f"❌ 데이터프레임 [{df_name}]의 전처리 코드 실행을 최종적으로 실패했습니다.")
                    break  # 재시도 후에도 실패하면 루프 종료

                attempt_count += 1  # 재시도 횟수 증가
    else:
        print(f"❌ 데이터프레임 [{df_name}]의 전처리 코드 실행이 취소되었습니다.")



=== 📊 EDA 결과: df_cust ===


,데이터프레임명,컬럼명,데이터 타입,count,mean,std,min,25%,75%,max,결측 개수,결측치 비율,고유값 개수,인스턴스(예제),범주형 분포
0,df_cust,고객ID,int64,120000,43126.17,33176.95,1.0,14297.25,69490.25,100000.00,0,0,15474,NaN,NaN
1,df_cust,나이,object,120000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,9,"['60대', '40대', '50대', '30대', '20대', '10세미만', '...","{'50대': 31.2, '40대': 30.53, '30대': 16.98, '20대..."
2,df_cust,성별,object,120000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2,"['여성', '남성']","{'여성': 52.44, '남성': 47.56}"
3,df_cust,수익자여부,int64,120000,0.89,0.31,0.0,1.00,1.00,1.00,0,0,2,NaN,NaN
4,df_cust,CB신용평점,float64,14118,316.21,280.35,0.0,95.16,457.66,1049.57,105882,88,12570,NaN,NaN



토큰 수: 12188
[LOG] df_cust 인사이트 생성 결과
  ### 인사이트 도출 및 데이터 정제 제안

1. **고객 ID 및 성별 관련**
   - `고객ID`는 고유 식별자로 사용되며, 결측치가 없습니다.
   - `성별`과 `성별코드`는 동일한 정보를 나타내는 것으로 보입니다. `성별코드`의 '-' 값은 불명확한 성별을 나타낼 수 있으므로, 해당 값을 '알 수 없음'으로 처리하거나 제거하는 것이 좋습니다.

2. **나이 및 보험 관련**
   - `나이`는 범주형 데이터로, 50대와 40대가 가장 많은 비율을 차지합니다. 이는 보험 상품의 주요 타겟 연령층을 나타낼 수 있습니다.
   - `보험연령`은 0부터 89까지의 값을 가지며, 0의 값은 비정상적인 데이터로 보입니다. 0의 값을 가진 데이터를 검토하여 수정하거나 제거해야 합니다.

3. **신용 관련**
   - `CB신용평점`과 `CB신용등급`의 결측치 비율이 각각 88%와 85%로 매우 높습니다. 이 데이터는 분석에 큰 영향을 미칠 수 있으므로, 결측치를 대체하거나 해당 컬럼을 제거하는 것을 고려해야 합니다.

4. **운전 및 직업 관련**
   - `운전코드명`의 결측치 비율이 45%로 높습니다. 결측치를 '운전안함'으로 대체하거나, 해당 컬럼을 분석에서 제외할 수 있습니다.
   - `직업분류명`의 결측치 비율이 40%로 높습니다. 결측치를 '알 수 없음'으로 대체하거나, 해당 컬럼을 분석에서 제외할 수 있습니다.

5. **채널 관련**
   - `방카채널Affluent고객여부`, `GA채널Affluent고객여부`, `자사설계사채널Affluent고객여부`는 모두 0으로만 구성되어 있어, 분석에 유의미한 정보를 제공하지 않습니다. 해당 컬럼은 제거하는 것이 좋습니다.

6. **금융 및 보험 관련**
   - `당월보험료자동대출잔액`의 결측치 비율이 92%로 매우 높습니다. 이 컬럼은 분석에서 제외하거나, 결측치를 대체할 방법을 찾아야 합니다.
   - `누적연금지급금액`, 

,고객ID,나이,성별,수익자여부,CB신용평점,CB신용등급,두낫콜여부,운전코드명,성별코드,피보험자여부,...,변액보유여부,변액최대납입회차,변액유지계약수,변액기납입보험료,변액종신CMIP,변액종신보유여부,변액종신최대납입회차,변액종신유지계약수,변액종신기납입보험료,기준년월
0,25226,60대,여성,1,NaN,NaN,1.0,승용차(자가용),02,1,...,0,11.09,0.0,6424.05,69074.36,0,3.15,0.0,539090.38,202405
1,95256,40대,남성,1,NaN,6.0,0.0,화물차(자가용),02,1,...,0,3.98,0.0,9592.97,2023.21,0,34.68,0.0,3338509.65,202405
2,14751,40대,여성,1,NaN,NaN,1.0,NaN,02,1,...,0,4.06,0.0,22093.94,11006.27,0,9.01,0.0,1356509.39,202405
3,4478,50대,여성,1,NaN,6.0,0.0,운전안함,01,1,...,0,8.71,0.0,6071.17,6473.26,0,1.02,0.0,16478801.52,202405
4,100000,50대,여성,0,245.26,NaN,NaN,운전안함,02,1,...,0,1.94,0.0,16569.67,2852.47,0,17.67,0.0,373537.44,202405


✅ 1차시도 데이터프레임 [df_cust]의 전처리 코드가 실행되었습니다! | 전처리 이전 데이터 = (120000, 116) | 전처리 이후 데이터 = (120000, 116)

=== 📊 EDA 결과: df_enroll_history ===


,데이터프레임명,컬럼명,데이터 타입,count,mean,std,min,25%,75%,max,결측 개수,결측치 비율,고유값 개수,인스턴스(예제),범주형 분포
123,df_enroll_history,고객ID,int64,28182,10021.89,5837.50,6.0,4917.0,15141.75,19998.0,0,0,6744,NaN,NaN
124,df_enroll_history,가입년월일,object,28182,NaN,NaN,NaN,NaN,NaN,NaN,0,0,5439,"['2019-05-09', '2012-02-07', '2018-05-23', '20...","{'2020-02-25': 0.06, '2023-07-28': 0.05, '2013..."
125,df_enroll_history,가입담보명,object,28182,NaN,NaN,NaN,NaN,NaN,NaN,0,0,27,"['상해입원일당', '질병사망', '뇌졸중진단', '상해종수술', '급성심근경색진단...","{'상해사망': 10.07, '질병사망': 9.85, '치매진단(경증)': 6.98..."
126,df_enroll_history,가입담보금액,int64,28182,52683917.25,70013490.69,10000.0,4000000.0,77000000.00,300000000.0,0,0,1135,NaN,NaN



토큰 수: 920
[LOG] df_enroll_history 인사이트 생성 결과
  ### 인사이트 도출 및 피드백

1. **고객ID (int64)**
   - 고객ID는 총 6744개의 고유값을 가지며, 이는 고객의 수를 나타냅니다. 데이터셋의 총 인스턴스 수는 28182개로, 고객당 평균적으로 약 4.18개의 가입 기록이 존재합니다. 이는 고객이 여러 상품에 가입했거나, 동일 상품에 대해 여러 번 가입했을 가능성을 시사합니다.
   - **피드백:** 고객ID는 중복 가입 여부를 확인하거나 고객별 가입 패턴을 분석하는 데 유용할 수 있습니다.

2. **가입년월일 (object)**
   - 가입년월일은 총 5439개의 고유값을 가지며, 이는 다양한 시점에 걸쳐 가입이 이루어졌음을 나타냅니다. 범주형 분포에서 특정 날짜에 가입이 집중된 경향은 보이지 않으며, 다양한 날짜에 고르게 분포되어 있습니다.
   - **피드백:** 가입년월일을 datetime 형식으로 변환하여 시간에 따른 가입 추세를 분석할 수 있습니다. 예를 들어, 특정 연도나 월에 가입이 증가하는 패턴을 찾을 수 있습니다.

3. **가입담보명 (object)**
   - 가입담보명은 총 27개의 고유값을 가지며, 이는 다양한 보험 상품이 존재함을 나타냅니다. 상해사망, 질병사망, 치매진단(경증) 등의 담보가 높은 비율로 가입되어 있습니다.
   - **피드백:** 특정 담보에 대한 가입 비율이 높은 이유를 분석하여, 고객의 니즈를 파악하고 상품 개발에 반영할 수 있습니다. 또한, 담보명을 카테고리형 데이터로 변환하여 분석 효율성을 높일 수 있습니다.

4. **가입담보금액 (int64)**
   - 가입담보금액의 평균은 약 5.27억 원이며, 최대값은 30억 원입니다. 이는 고객의 보험 가입 금액이 매우 다양함을 나타냅니다.
   - **피드백:** 가입담보금액의 분포를 시각화하여 이상치 여부를 확인할 수 있습니다. 또한, 금액을 기준으로 고객 세그먼트를 나누어 분석할 수 있습니다.

#

,고객ID,가입년월일,가입담보명,가입담보금액
0,6,2019-05-09,상해입원일당,30000
1,6,2012-02-07,질병사망,96000000
2,6,2018-05-23,뇌졸중진단,18000000
3,6,2020-02-27,상해종수술,68000000
4,6,2016-06-26,급성심근경색진단,13000000


✅ 1차시도 데이터프레임 [df_enroll_history]의 전처리 코드가 실행되었습니다! | 전처리 이전 데이터 = (28182, 4) | 전처리 이후 데이터 = (28182, 4)
